In [1]:
from LSTM_Attn import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
valid_df = build_valid_df(CSV_PATH, IMG_ROOT)
if valid_df.empty:
    print("[WARN] No valid rows found; check paths and PNG conversion.")
labels_as_str = valid_df[TEXT_COL].astype(str).tolist()
tokenizer = build_tokenizer_from_labels(labels_as_str)
pad_id = getattr(tokenizer, "pad_token_id", 0)
bos_id = getattr(tokenizer, "bos_token_id", 1)
eos_id = getattr(tokenizer, "eos_token_id", 2)
tf = dino_image_transform(img_size=516)
ds = CheXpertDataset(img_root=IMG_ROOT, csv=valid_df, transform=tf, text_col=TEXT_COL)
collate_fn = CaptionCollate(tokenizer, pad_id)
is_windows = os.name == "nt"
num_workers = 0 if is_windows else 2
persistent_workers = False if num_workers == 0 else True
loader = DataLoader(
    ds,
    batch_size=8,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=persistent_workers,
    collate_fn=collate_fn
)
train_ds = torch.utils.data.Subset(ds, range(0, 80))#int(len(ds)*.8)))
valid_ds = torch.utils.data.Subset(ds, range(80, 160))#int(len(ds)*.8), len(ds)))
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_ds, batch_size=8, shuffle=False, collate_fn=collate_fn)
D_IMG = 384
model = DinoLSTMAttnCaptioner(
    vocab_size=tokenizer.vocab_size,
    d_img=D_IMG,
    d_h=512,
    pad_id=pad_id,
    dino_model_id="facebook/dinov3-vits16-pretrain-lvd1689m",
    freeze_dino=True,
).to(device)
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4, weight_decay=1e-2
)


Using device: cuda
[INFO] Kept 47494/223462 rows with existing PNGs under C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\PNG


In [3]:
for epoch in range(100):
    slice_train_loader = islice(train_loader, 10)
    slice_valid_loader = islice(valid_loader, 10)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=10, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=10)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")
test_loader_sliced = iter(valid_loader)
with torch.no_grad():
    for batch in test_loader_sliced:
        pixel_values, ids_loader, paths, raw_labels = batch
        pixel_values = pixel_values.to(device)
        gen_ids = model.generate(
            pixel_values=pixel_values,
            bos_id=bos_id, eos_id=eos_id,
            max_new_tokens=50, top_p=0.8, temperature=0.9, greedy=False
        )
        print("Predictions test:")
        for i in range(gen_ids.size(0)):
            print(f"\nTEST GEN {i+1}:", tokenizer.decode(gen_ids[i].tolist()))
            print(f"TEST TARGET {i+1}:", tokenizer.decode(ids_loader[i].tolist()))
        # Free batch memory
        del pixel_values, ids_loader, paths, raw_labels, gen_ids
        torch.cuda.empty_cache()
        break

Evaluating: 100%|██████████| 10/10 [00:05<00:00,  1.79it/s]


Epoch 1: Train Loss=9.4246, PPL=15369.86 | Val Loss=8.1643, Val PPL=3575.08


Evaluating: 100%|██████████| 10/10 [00:05<00:00,  1.79it/s]


Epoch 2: Train Loss=7.2939, PPL=1726.83 | Val Loss=6.4971, Val PPL=698.94


Evaluating: 100%|██████████| 10/10 [00:05<00:00,  1.80it/s]


...

Epoch 98: Train Loss=1.3796, PPL=3.97 | Val Loss=5.1346, Val PPL=208.29


Evaluating: 100%|██████████| 10/10 [00:05<00:00,  1.75it/s]


Epoch 99: Train Loss=1.3795, PPL=3.97 | Val Loss=5.1310, Val PPL=208.23


Evaluating: 100%|██████████| 10/10 [00:05<00:00,  1.74it/s]


Epoch 100: Train Loss=1.3791, PPL=3.97 | Val Loss=5.1423, Val PPL=210.00
Predictions test:

TEST GEN 1: 1. supine frontal view of the chest demonstrates interval removal of the enteric tube. the remaining right ij catheter and surgical materials are stable. 2. the heart is moderately enlarged and mitral annular calcification as well as
TEST TARGET 1: 1. stable right internal jugular central venous catheter, prosthetic heart valve, and median sternotomy wires. 2. loculated right pleural effusion, low lung volumes in the right hemithorax, and mild cardiomegaly are unchanged. 3. slight increase in diffuse interstitial opacities, likely reflecting underlying mild pulmonary edema increased.

TEST GEN 2: 1. supine frontal view of the chest demonstrates interval removal of the enteric tube. the remaining right ij catheter and surgical materials are stable. 2. the heart is moderately enlarged and mitral annular calcification as well as
TEST TARGET 2: 1. stable right internal jugular central ve

In [4]:
# Installs you might need:
# !pip install -U nltk rouge-score bert-score f1chexbert requests appdirs

import os, shutil, requests
import torch, numpy as np
from appdirs import user_cache_dir
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import wordpunct_tokenize
from rouge_score import rouge_scorer

# ---- Optional imports (guarded) ----
try:
    from bert_score import score as bertscore_score
except Exception:
    bertscore_score = None

# ---- CheXbert cache utils ----
CHEXBERT_URLS = [
    # Official Stanford Box link (may redirect / require retries)
    "https://stanfordmedicine.box.com/shared/static/c3stck6w6dol3h36grdc97xoydzxd7w9",
    # Hugging Face mirror (often more reliable)
    "https://huggingface.co/StanfordAIMI/RRG_scorers/resolve/main/chexbert.pth",
]

def default_chexbert_cache_dir():
    # Matches f1chexbert's default on Windows:
    # C:\Users\<YOU>\AppData\Local\chexbert\chexbert\Cache
    return os.path.join(user_cache_dir("chexbert", "chexbert"), "Cache")

def chexbert_ckpt_path(cache_dir=None, filename="chexbert.pth"):
    cache_dir = cache_dir or default_chexbert_cache_dir()
    return os.path.join(cache_dir, filename)

def delete_chexbert_weights(cache_dir=None, filename="chexbert.pth"):
    """Delete existing CheXbert weights if present."""
    path = chexbert_ckpt_path(cache_dir, filename)
    if os.path.exists(path):
        try:
            os.remove(path)
            print(f"[CheXbert] Deleted old weights at: {path}")
        except Exception as e:
            print(f"[CheXbert] Could not delete {path}: {e}")
    else:
        print("[CheXbert] No existing weights to delete.")

def ensure_chexbert_weights(cache_dir=None, filename="chexbert.pth", force=False, min_bytes=10_000_000):
    """Ensure weights exist. If force=True, redownload even if file exists."""
    cache_dir = cache_dir or default_chexbert_cache_dir()
    os.makedirs(cache_dir, exist_ok=True)
    dst = os.path.join(cache_dir, filename)

    if not force and os.path.exists(dst) and os.path.getsize(dst) > min_bytes:
        return dst  # looks valid

    # (Re)download
    for url in CHEXBERT_URLS:
        try:
            print(f"[CheXbert] Downloading weights from: {url}")
            with requests.get(url, stream=True, timeout=120, allow_redirects=True) as r:
                r.raise_for_status()
                tmp = dst + ".tmp"
                with open(tmp, "wb") as f:
                    for chunk in r.iter_content(chunk_size=1024 * 1024):
                        if chunk:
                            f.write(chunk)
                os.replace(tmp, dst)
            if os.path.getsize(dst) > min_bytes:
                print(f"[CheXbert] Weights saved to: {dst}")
                return dst
            else:
                print(f"[CheXbert] Downloaded file too small ({os.path.getsize(dst)} bytes), trying next mirror...")
        except Exception as e:
            print(f"[CheXbert] Download failed from {url}: {e}")
    return None

# ---- Delete then (re)download weights ----
delete_chexbert_weights()
_ckpt_path = ensure_chexbert_weights(force=True)

# ---- Try to load CheXbert evaluator ----
try:
    from f1chexbert import F1CheXbert
    _chexbert = F1CheXbert() if _ckpt_path else None
    if _chexbert is None:
        print("[Info] CheXbert unavailable (no weights). Place chexbert.pth in the cache dir above.")
except Exception as e:
    _chexbert = None
    print(f"[Info] CheXbert not available: {e}")

# ---- Helpers ----
def tok(s: str):
    return [w for w in wordpunct_tokenize(s.lower()) if w.strip()]

_warned_bert = False
def safe_bertscore(pred: str, tgt: str) -> float:
    global _warned_bert
    if bertscore_score is None:
        if not _warned_bert:
            print("[Info] bert-score not installed/available; BERTScore will be NaN. `pip install bert-score`.")
            _warned_bert = True
        return float("nan")
    P, R, F = bertscore_score([pred], [tgt], lang="en", rescale_with_baseline=True)
    return float(F.mean().item())

# ---- Metric setup ----
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
smooth = SmoothingFunction().method1

bleu_list, rougeL_list, meteor_list, bert_list = [], [], [], []
gens_all, tgts_all = [], []

# ===== Evaluation loop (single batch), assumes: valid_loader, model, tokenizer, device, bos_id, eos_id =====
test_loader_sliced = iter(valid_loader)
with torch.no_grad():
    for batch in test_loader_sliced:
        pixel_values, ids_loader, paths, raw_labels = batch
        pixel_values = pixel_values.to(device)

        gen_ids = model.generate(
            pixel_values=pixel_values,
            bos_id=bos_id, eos_id=eos_id,
            max_new_tokens=50, top_p=0.8, temperature=0.9, greedy=False
        )

        print("Predictions test:")
        for i in range(gen_ids.size(0)):
            # Skip special tokens to avoid artifacts
            gen_text = tokenizer.decode(gen_ids[i].tolist())
            tgt_text = tokenizer.decode(ids_loader[i].tolist())

            print(f"\nTEST GEN {i+1}:", gen_text)
            print(f"TEST TARGET {i+1}:", tgt_text)

            # BLEU
            bleu = sentence_bleu([tgt_text.split()], gen_text.split(), smoothing_function=smooth)
            # ROUGE-L (F)
            rougeL = scorer.score(tgt_text, gen_text)['rougeL'].fmeasure
            # METEOR (token lists)
            try:
                meteor = meteor_score([tok(tgt_text)], tok(gen_text))
            except TypeError:
                meteor = meteor_score([tgt_text], gen_text)
            # BERTScore
            bert = safe_bertscore(gen_text, tgt_text)

            print(f"BLEU: {bleu:.4f} | ROUGE-L: {rougeL:.4f} | METEOR: {meteor:.4f} | BERTScore(F1): {bert:.4f}")

            bleu_list.append(bleu); rougeL_list.append(rougeL); meteor_list.append(meteor); bert_list.append(bert)
            gens_all.append(gen_text); tgts_all.append(tgt_text)

        def nanmean(x):
            arr = np.array(x, dtype=float)
            return float(np.nanmean(arr)) if arr.size else float("nan")

        print("\n--- Batch means (text metrics) ---")
        print(f"BLEU: {nanmean(bleu_list):.4f} | ROUGE-L: {nanmean(rougeL_list):.4f} | "
              f"METEOR: {nanmean(meteor_list):.4f} | BERTScore(F1): {nanmean(bert_list):.4f}")

        # ---- CheXbert (batch clinical metric) ----
        if _chexbert is not None and gens_all and len(gens_all) == len(tgts_all):
            try:
                chex_accuracy, chex_acc_not_avg, chex_report, chex_report_5 = _chexbert(
                    hyps=gens_all, refs=tgts_all
                )
                print("\n--- CheXbert (batch) ---")
                print(f"CheXbert macro F1 (called 'accuracy' in pkg): {float(chex_accuracy):.4f}")
                if isinstance(chex_report_5, dict):
                    top5 = ', '.join([f"{k}: {v:.3f}" for k, v in chex_report_5.items()])
                    print(f"Top-5 precision [Cardiomegaly, Edema, Consolidation, Atelectasis, Pleural Effusion]: {top5}")
            except Exception as e:
                print(f"[Warn] CheXbert failed to run: {e}")

        # Cleanup
        del pixel_values, ids_loader, paths, raw_labels, gen_ids
        torch.cuda.empty_cache()
        break


[CheXbert] Deleted old weights at: C:\Users\emman\AppData\Local\chexbert\chexbert\Cache\Cache\chexbert.pth
[CheXbert] Downloading weights from: https://stanfordmedicine.box.com/shared/static/c3stck6w6dol3h36grdc97xoydzxd7w9
[CheXbert] Download failed from https://stanfordmedicine.box.com/shared/static/c3stck6w6dol3h36grdc97xoydzxd7w9: 404 Client Error: Not Found for url: https://stanfordmedicine.app.box.com/public/static/c3stck6w6dol3h36grdc97xoydzxd7w9
[CheXbert] Downloading weights from: https://huggingface.co/StanfordAIMI/RRG_scorers/resolve/main/chexbert.pth
[CheXbert] Weights saved to: C:\Users\emman\AppData\Local\chexbert\chexbert\Cache\Cache\chexbert.pth
[Info] CheXbert not available: [Errno 2] No such file or directory: 'C:\\Users\\emman\\AppData\\Local\\chexbert\\chexbert\\Cache\\chexbert.pth'
Predictions test:

TEST GEN 1: 1. interval appearance of endotracheal tube which is 4 cm above the carina. a defibrillator pad overlies the left hemithorax. 2. right lower lobe opacifica